<a href="https://colab.research.google.com/github/cherest555/CLASS_AI4DL/blob/main/baseline_distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data prep

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [7]:
train = pd.read_csv('/content/sample_data/train.csv')
test = pd.read_csv('/content/sample_data/test.csv')
sample = pd.read_csv('/content/sample_data/sample_submission.csv')

In [8]:
print(train.shape, test.shape)
train.sample(10, random_state=26)

(7613, 5) (3263, 4)


,id,keyword,location,text,target
2943,4232,drowned,NaN,I want to see my @AustinPearcy22 so bad its no...,0
3723,5293,fear,Alliston Ontario,@ScottDPierce @billharris_tv @HarrisGle @Beeze...,0
4809,6845,loud%20bang,NaN,need to work in an office I can bang all my fa...,0
2517,3617,desolation,St. Louis Mo.,Wow! I just won this for free The Hobbit: Deso...,0
7591,10846,NaN,NaN,Heat wave warning aa? Ayyo dei. Just when I pl...,1
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0
4359,6193,hijacker,"Sarasota, FL",Remove the http://t.co/Xxj2B4JxRt and Linkury ...,0
2356,3391,demolition,"Funtua, Nigeria",Kaduna Begins Demolition Of Buildings On Govt ...,1
2074,2976,dead,dundalk ireland,@emmerdale is Ross really dead?? #AskCharley,0
5673,8096,rescued,Nigeria,4 kidnapped ladies rescued by police in Enugu ...,1


In [9]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [11]:
df_train.shape

(7613, 5)

In [12]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [13]:
df_test.shape

(3263, 4)

Get rid of twitter handles (@sometext98), take the lowercase text, and add in an extra feature for training

In [16]:
import re

# Preprocess text
def preprocess(df):
    df_new = df.copy(deep=True)
    
    # Get rid of twitter handles, and take lowercase text
    df_new['text'] = df.apply(lambda row: re.sub('@[A-z0-9]', '', row['text']).lower(), axis=1)
    
    # Training feature - keyword + text
    df_new['text_w_kword'] = df_new.apply(lambda row: 'keyword: ' + str(row['keyword']) + '. '+ str(row['text']), axis=1)
    return df_new

train_prep = preprocess(train)
test_prep = preprocess(test)

train_prep[40:60]

,id,keyword,location,text,target,text_w_kword
40,59,ablaze,Live On Webcam,check these out: http://t.co/roi2nsmejj http:/...,0,keyword: ablaze. check these out: http://t.co/...
41,61,ablaze,NaN,on the outside you're ablaze and alive\nbut yo...,0,keyword: ablaze. on the outside you're ablaze ...
42,62,ablaze,milky way,had an awesome time visiting the cfc head offi...,0,keyword: ablaze. had an awesome time visiting ...
43,63,ablaze,NaN,soooo pumped for ablaze ???? outhridgelife,0,keyword: ablaze. soooo pumped for ablaze ???? ...
44,64,ablaze,NaN,i wanted to set chicago ablaze with my preachi...,0,keyword: ablaze. i wanted to set chicago ablaz...
45,65,ablaze,NaN,i gained 3 followers in the last week. you? kn...,0,keyword: ablaze. i gained 3 followers in the l...
46,66,ablaze,"GREENSBORO,NORTH CAROLINA",how the west was burned: thousands of wildfire...,1,keyword: ablaze. how the west was burned: thou...
47,67,ablaze,NaN,building the perfect tracklist to life leave t...,0,keyword: ablaze. building the perfect tracklis...
48,68,ablaze,Live On Webcam,check these out: http://t.co/roi2nsmejj http:/...,0,keyword: ablaze. check these out: http://t.co/...
49,71,ablaze,England.,first night with retainers in. it's quite weir...,0,keyword: ablaze. first night with retainers in...


Split training data into train and validation data (used to stop overfitting).

In [17]:
from sklearn.model_selection import train_test_split

# Split data
X_train, X_valid, y_train, y_valid = train_test_split(train_prep['text_w_kword'], # Inputs
                                                    train_prep['target'], # Labels
                                                    test_size=0.1, 
                                                    random_state=1)

In [47]:
# Load tokenizer
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast(
    tokenizer_file='/content/sample_data/distilbert/tokenizer.json',
    vocab_file='/content/sample_data/distilbert/vocab.txt'
)

# Get encodings
# For this dataset, the largest set of tokens was 88 long, so max_length = 100 is fine here
# Padding means for any dataset shorter than 100, it adds 0s to reach 100 in length
# Truncation does the opposite, it'll cut any tokens past 100 - not needed here, but good practice!
train_encodings = tokenizer(list(X_train), truncation=True, padding='max_length', max_length=100)
valid_encodings = tokenizer(list(X_valid), truncation=True, padding='max_length', max_length=100)

In [48]:
# Convert to dataset object
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train.values.astype('float32').reshape((-1,1)) # To make it 2 dimensional
))

valid_dataset = tf.data.Dataset.from_tensor_slices((
    dict(valid_encodings),
    y_valid.values.astype('float32').reshape((-1,1)) # To make it 2 dimensional
))

train_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(100,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(100,), dtype=tf.int32, name=None)}, TensorSpec(shape=(1,), dtype=tf.float32, name=None))>

Modelling

In [49]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_loss', 
                   verbose=1, # Prints outputs
                   patience=4, # Waits 4 epochs for an improvement
                   restore_best_weights=True) # Restores model back to best epoch

In [50]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay
batch_size = 64 
num_epochs = 15
# The number of training steps in total is the number of samples in the dataset, divided by the batch size 
# then multiplied by the total number of epochs
num_train_steps = (X_train.shape[0] // batch_size) * num_epochs

# This is actually a linear decay from 5x10^-5 to 1x10^-5, not actually polynomial!
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, # Start point
    end_learning_rate=1e-5, # End point
    decay_steps=num_train_steps 
    )

from tensorflow.keras.optimizers import Adam
new_opt = Adam(learning_rate=lr_scheduler)

In [51]:
from keras import backend as K

def f1_score(true, pred): 

    ground_positives = K.sum(true, axis=0) + K.epsilon()       # = TP + FN
    pred_positives = K.sum(pred, axis=0) + K.epsilon()         # = TP + FP
    true_positives = K.sum(true * pred, axis=0) + K.epsilon()  # = TP
    
    precision = true_positives / pred_positives 
    recall = true_positives / ground_positives

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    
    return f1

In [52]:
# Fine tune with native Tensorflow
from transformers import TFDistilBertForSequenceClassification

# Load model, set up for regression
model = TFDistilBertForSequenceClassification.from_pretrained('/kaggle/input/huggingface-bert-variants/distilbert-base-uncased/distilbert-base-uncased/',
                                                              num_labels=2)

model.compile(
    optimizer=new_opt,
    # Very important - model outputs logits, not probabilities!
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
)

history = model.fit(train_dataset.batch(batch_size),
                    validation_data=valid_dataset.batch(batch_size),
                    epochs=num_epochs,
                    callbacks=[es])

OSError: ignored

Predictions

In [ ]:
test_encodings = tokenizer(list(test_prep['text_w_kword']), truncation=True, padding='max_length', max_length=100)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings)
))

In [ ]:
test_preds = model.predict(test_dataset.batch(1))

In [ ]:
test_prep

In [ ]:
test_preds.logits[:40]

In [ ]:
class_preds = np.argmax(test_preds.logits, axis=1)
class_preds

In [ ]:
valid_preds = model.predict(valid_dataset.batch(batch_size))

In [ ]:
valid_class_preds = np.argmax(valid_preds.logits, axis=1)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_valid, valid_class_preds))

In [ ]:
df_submission = pd.DataFrame({'id':test['id'].values,
                             'target':class_preds})

df_submission

In [ ]:
df_submission.to_csv('submission.csv', index=False)